In [1]:
exec(open("../header.py").read())

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Ridge

In [4]:
train = pd.read_csv(data_path/"salary-train.csv")
train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [6]:
test = pd.read_csv(data_path/"salary-test-mini.csv")
test.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [18]:
def normalize_inplace(df):
    df["FullDescription"] = df["FullDescription"].apply(lambda x: x.lower())
    df["FullDescription"] = df["FullDescription"].replace("[^a-z0-9]", " ", regex=True)
    df["LocationNormalized"] = df["LocationNormalized"].fillna("nan")
    df["ContractTime"] = df["ContractTime"].fillna("nan")

In [19]:
normalize_inplace(train)
normalize_inplace(test)

In [33]:
train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse rgn nursing home for young...,Sutton Coldfield,nan,20355


Vectorize descriptions

In [21]:
tfidf_vectorizer = TfidfVectorizer(min_df=5)

In [23]:
full_descr_vec = tfidf_vectorizer.fit_transform(train["FullDescription"])

In [24]:
full_descr_test_vec = tfidf_vectorizer.transform(test["FullDescription"])

Vectorize `LocationNormalized` and `ContractTime`

In [39]:
dict_vectorizer = DictVectorizer()

In [44]:
categ_vec = dict_vectorizer.fit_transform(train[["LocationNormalized", "ContractTime"]].to_dict("records"))
categ_vec_test = dict_vectorizer.transform(test[["LocationNormalized", "ContractTime"]].to_dict("records"))

In [50]:
X = sp.sparse.hstack([
    full_descr_vec,
    categ_vec,
])

In [51]:
X_test = sp.sparse.hstack([
    full_descr_test_vec,
    categ_vec_test,
])

In [52]:
y = train["SalaryNormalized"]
y_test = test["SalaryNormalized"]

In [53]:
ridge_reg = Ridge(alpha=1, random_state=241)

In [54]:
ridge_reg.fit(X, y)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=241, solver='auto', tol=0.001)

In [58]:
@answer
def ridge_reg_salary(fout):
    res = ridge_reg.predict(X_test)
    fout.write(f"{res[0]:.2f} {res[1]:.2f}")
ridge_reg_salary()

*** OUTPUT (output/01_ridge_reg_salary.dat) ***
56555.62 37188.32
